# Booking scraping: Stata-ready export + DiD with city/date fixed effects



## Why this is needed
Stata fails with `r(5101)` because the `text` column contains embedded newlines and problematic quoting. We avoid that by excluding `text` from the Stata import.


![bse_logo_textminingcourse](https://bse.eu/sites/default/files/bse_logo_small.png)

# Introduction to Text Mining and Natural Language Processing

Professor: Hannes Mueller
TA: Margherita Philipp

DiD with scraped data

This notebook:
1. Loads the original CSV (even if it contains multiline `text`).
2. Runs the simple DiD in Python with city and date fixed effects.

In [ ]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

## 0) Set paths and load data

Update `CSV_PATH` to your local file path. Outputs go to `OUT_DIR`.

In [ ]:
CSV_PATH = Path(r"data/booking_scraping_combined.csv")
OUT_DIR = Path(r"output")

OUT_DIR.mkdir(parents=True, exist_ok=True)
print("CSV_PATH:", CSV_PATH)
print("OUT_DIR:", OUT_DIR)

We rely on pandas’ CSV parser, which can handle multiline quoted strings more robustly than Stata.

In [ ]:
df = pd.read_csv(CSV_PATH, encoding="utf-8")
print(df.shape)
df.head()

# (20755, 11)

## 1) Text features

Generated for all groups - can subset later

### Dictionary terms

In [ ]:
luxury_terms = [
]

### Clean data

In [ ]:
df.group.unique()

In [ ]:
df["text_clean"] = (
    df["text"]
    .str.lower()
    .str.replace(r"[^\w\s\.-]", " ", regex=True)  #[^\w\s\.-] vs [^\w\s\]
    .str.replace(r"\s+", " ", regex=True)
    .str.strip()
)

In [ ]:
df[['text', 'text_clean']].head(10)

In [ ]:
df.loc[df.text.isna()]

In [ ]:
df = df.dropna(subset=["text", "price", "treatCity", "treatPeriod", "city", "date"]).copy()

### Applying the dictionary - with speed

In [ ]:
import re   

# compile once
luxury_pattern1 = re.compile(r"\b(" + "|".join(map(re.escape, luxury_terms)) + r")\b")
luxury_pattern2 = re.compile(r"\b(" + "|".join(map(re.escape, luxury_terms)) + r")\b", re.IGNORECASE)

In [ ]:
# choose version of text and pattern

text_col = "text_clean" # "text_clean text
luxury_pattern = luxury_pattern1

In [ ]:
luxury_pattern

In [ ]:
%%timeit

## OPTION A: python lopp 

luxury_count = []

for txt in df[text_col]:
    # if not isinstance(text, str):
    #     text = "" if text is None else str(text)  # or just ""
    luxury_count.append(len(luxury_pattern.findall(txt)))

df["luxury_count_loop"] = luxury_count

# for group 1
# text clean + pattern 1: 108 ms ± 2.94 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
# text raw   + pattern 2: 285 ms ± 3.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

# for all groups
# text clean + pattern 1: 315 ms ± 9.89 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
# text raw   + pattern 2: 825 ms ± 4.47 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

In [ ]:
%%timeit

## OPTION B: loop inside a function + .apply()

def count_luxury(txt: str) -> int:
    return len(luxury_pattern.findall(txt))

df["luxury_count_apply"] = df[text_col].apply(count_luxury)

# group 1:
# text clean + pattern 1: 106 ms ± 1.58 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
# text raw   + pattern 2: 285 ms ± 8.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

# for all groups
# text clean + pattern 1: 316 ms ± 10.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
# text raw   + pattern 2: 858 ms ± 16.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

In [ ]:
%%timeit

## OPTION C: Vectorized regex approach

df["luxury_count_vec"] = df[text_col].str.count(luxury_pattern)

# group 1: 
# text clean + pattern 1: 104 ms ± 1.74 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
# text raw   + pattern 2: 280 ms ± 1.57 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

# for all groups
# text clean + pattern 1: 308 ms ± 2.03 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
# text raw   + pattern 2: 884 ms ± 22.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

### Inspecting matches

In [ ]:
%%timeit
df["luxury_matches"] = df["text_clean"].str.findall(luxury_pattern)
df["luxury_count"] = df["luxury_matches"].str.len()

# all groups
# clean & 1: 317 ms ± 9.43 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

In [ ]:
# compare counts
df[['text','text_clean','luxury_count_loop', 'luxury_count_apply', 'luxury_count_vec']]

# check that none differ
df.loc[(df['luxury_count_apply'] != df['luxury_count_apply'])]

# inspect matches
df[['text','text_clean','luxury_matches', 'luxury_count']]

### Normalise by description length

In [ ]:
# description length
df["n_words"] = df["text_clean"].str.split().str.len()

# normalise
df["luxury_density"] = df["luxury_count"] / df["n_words"]

# quick inspection
df[["luxury_count", "luxury_density"]].describe()


In [ ]:
# quick visuals of distributions

bin_number = 40

df['luxury_count'].plot(kind='hist',
                        bins=bin_number, title='Distribution of luxury counts')
plt.show()

df['luxury_density'].plot(kind='hist',
                          bins=bin_number, title='Distribution of luxury nomalised')
plt.show()

In [ ]:
# binarisation suggestions
cut_off = 0.013
print("cut-off = ", cut_off)
# print("<= cut-off " , len(df.loc[df['luxury_density']<=cut_off]))
# print("> cut-off ", len(df.loc[df['luxury_density']>cut_off]))

df['luxury_bin'] = (df['luxury_density'] > cut_off).astype(int)
df['luxury_bin'].value_counts()


In [ ]:
df[['luxury_bin', 'luxury_count', 'luxury_density']]

## 2) Check Parallel trends

For a more robust analysis: do an event study
- event-time variable
- could be nonparametric binned event-study plot or regression-based event study

In [ ]:
df.sample (5)

### Select a subset of data and inspect hotel numbers and shares

In [ ]:
group = 4 # 1 (impact), 3 (no impact), 4 & 5 (less good data?)

# check what share of observations are in the treated city and which periods are in the treated period 
df_chosen = df.loc[df['group']==group]
df_chosen.date.unique()
df_gb = df_chosen.groupby('date').agg({'treatCity':'mean',
                               'treatPeriod':'mean'}).reset_index()
df_gb.sort_values("date", inplace=True)

# (un)treated dates 
dates_0 = df_gb.loc[df_gb['treatPeriod'] == 0, 'date']
dates_1 = df_gb.loc[df_gb['treatPeriod'] == 1, 'date']

# save for later - NB works less well for group 1 as no post-treatment control
first_date_0 = dates_0.iloc[0]
last_date_0  = dates_0.iloc[-1]
date_treat_1 = dates_1.iloc[0]

df_gb

In [ ]:
print("Share of luxurious hotels in treatment: ", len(df_chosen.loc[(df_chosen['treatPeriod']==1) & (df_chosen['luxury_bin']==1)])/ len(df_chosen.loc[df_chosen['treatPeriod']==1]))
print("Share of luxurious hotels in control: ", len(df_chosen.loc[(df_chosen['treatPeriod']==0) & (df_chosen['luxury_bin']==1)])/ len(df_chosen.loc[df_chosen['treatPeriod']==0]))

In [ ]:
shares = (
    df_chosen
    .groupby('treatPeriod')['luxury_bin']
    .mean()
)

n_obs = (
    df_chosen
    .groupby(['treatPeriod', 'date'])['luxury_bin']
    .count()
)

print("Share of luxurious hotels in treatment:", round(shares.loc[1], 4))
print("Share of luxurious hotels in control:", round(shares.loc[0], 4))
print("___")
# change date
print("Number of luxurious hotels in treatment:", n_obs.loc[(1, date_treat_1)])
print("Number of luxurious hotels in (pre-)control:", n_obs.loc[(0, first_date_0)])
print("___")
print("Number of luxurious hotels in treatment:", n_obs.loc[(1, date_treat_1)])
print("Number of luxurious hotels in (after-)control:", n_obs.loc[(0, last_date_0)])

In [ ]:
set_lux_treat = set(df_chosen.loc[(df_chosen['treatPeriod']==1) & (df_chosen['luxury_bin']==1)]["hotel"])
set_lux_contr = set(df_chosen.loc[(df_chosen['treatPeriod']==0) & (df_chosen['luxury_bin']==1)]["hotel"])

# what does each of these show?
set_lux_treat - set_lux_contr
set_lux_contr - set_lux_treat

In [ ]:
# Ensure date is datetime
df_chosen['date'] = pd.to_datetime(df_chosen['date'])

# Average price by date and treatment status
df_avg = (
    df_chosen
    .groupby(['date', 'treatCity'], as_index=False)['price']
    .mean())

df_treated = df_avg[df_avg['treatCity'] == 1]
df_control = df_avg[df_avg['treatCity'] == 0]

plt.figure(figsize=(10, 6))

# Treated: points + thin line
plt.plot(
    df_treated['date'],
    df_treated['price'],
    marker='o',
    linestyle='-',
    linewidth=1,
    markersize=5,
    label=f'Treated: {df_chosen.event_city.unique()[0]}' #'Treated cities'
)

# Control: points + thin line
plt.plot(
    df_control['date'],
    df_control['price'],
    marker='o',
    linestyle='-',
    linewidth=1,
    markersize=5,
    label=f'Control: {df_chosen.control_city.unique()[0]}' # Control cities'
)

# Treatment dates
treatment_dates = df_chosen.loc[df_chosen['treatPeriod']==1].date.unique()
for d in treatment_dates:
    plt.axvline(d, linestyle='--', alpha=0.5)

plt.xlabel('Date')
plt.ylabel('Average price')
plt.title(f'Average hotel prices: treated ({df_chosen.event.unique()[0]}) vs control city')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Ensure date is datetime
df_chosen['date'] = pd.to_datetime(df_chosen['date'])

# Average price by date, treatment status, and luxury bin
df_avg = (
    df_chosen
    .groupby(['date', 'treatCity', 'luxury_bin'], as_index=False)['price']
    .mean()
)

plt.figure(figsize=(10, 6))

# Plot 4 lines
for treat, treat_label in [(1, 'Treated'), (0, 'Control')]:
    for lux, lux_label in [(1, 'Luxury'), (0, 'Non-luxury')]:
        tmp = df_avg[(df_avg['treatCity'] == treat) & (df_avg['luxury_bin'] == lux)]

        # Style controls
        linestyle = '-' if lux == 1 else '--'
        color = 'orange' if treat == 1 else 'blue'

        plt.plot(
            tmp['date'],
            tmp['price'],
            marker='o',
            linestyle=linestyle,
            linewidth=1,
            markersize=5,
            color=color,
            label=f'{treat_label} — {lux_label}'
        )


# Treatment dates
treatment_dates = df_chosen.loc[df_chosen['treatPeriod'] == 1, 'date'].dropna().unique()
for d in treatment_dates:
    plt.axvline(d, linestyle='--', alpha=0.5)

plt.xlabel('Date')
plt.ylabel('Average price')
plt.title(f'Average hotel prices (treated vs control) split by binarised luxury density')
plt.legend()
plt.tight_layout()
plt.show()


## 3) Run DiD in Python

"Simple" specification: `ln(price) ~ τ (treatCity*treatPeriod) + city FE + date FE`

### Transform target

In [ ]:
import numpy as np
import statsmodels.api as sm
from statsmodels.stats.sandwich_covariance import cov_cluster_2groups

# NB NaN values already dropped
df_ln = df.copy()
df_ln["price"] = pd.to_numeric(df_ln["price"], errors="coerce")
df_ln = df_ln.dropna(subset=["price"]) #(already done)
df_ln["lnprice"]=np.log(df_ln["price"]) 

In [ ]:
# decide/ double check which groups' data we are working with
df_py = df_ln.loc[df_ln.group.isin([3])] # 1,3,4,5

df_py.date.unique()
df_py.group.unique()

### Run OLS on residuals after “absorbing” fixed effects via demeaning

 - Instead of adding thousands of dummies, you “within-transform” both _ln(price)_ and _treatment_ using `multiway_demean`.
 - No intercept needed because residualized variables have mean ~0.
- Estimate tau and check different error structures.

In [ ]:
# function for demeaning
def multiway_demean(v, *groups, max_iter=200, tol=1e-10):
    """
    Iteratively residualize v on multiple categorical fixed effects (groups).
    v: (n,) array-like
    groups: one or more (n,) arrays of group ids (int/str ok)
    """
    v = np.asarray(v, dtype=float)
    out = v.copy()

    # Ensure groups are 1D arrays
    G = [np.asarray(g) for g in groups]
    n = len(out)
    for g in G:
        if g.shape[0] != n:
            raise ValueError("All group arrays must have same length as v.")

    for _ in range(max_iter):
        old = out.copy()

        for g in G:
            # Map group labels to consecutive ints
            _, inv = np.unique(g, return_inverse=True)
            sums = np.bincount(inv, weights=out)
            cnts = np.bincount(inv)
            means = sums / cnts
            out = out - means[inv]

        if np.max(np.abs(out - old)) < tol:
            break

    return out

In [ ]:
# define some fixed effects - numeric FE codes
g_city = df_py["city"].astype("category").cat.codes.to_numpy().astype(int)
g_date = df_py["date"].astype("category").cat.codes.to_numpy().astype(int)
g_hotel = df_py["hotel"].astype("category").cat.codes.to_numpy().astype(int)

In [ ]:
# "Simple": Treatment = TreatCity × TreatPeriod
df_py["treatment"] = df_py["treatCity"] * df_py["treatPeriod"]

# Within-transform (absorb Fiexed Effects: choose which ones) g_city g_date g_hotel 
y = multiway_demean(df_py["lnprice"].to_numpy(), g_city, g_date)
x = multiway_demean(df_py["treatment"].to_numpy(), g_city, g_date)

# OLS on residualized variables (no intercept)
res_simple = sm.OLS(y, x).fit()

res_simple.params

NB we now have two coefficients

ln(price)=β1​⋅treatment+β2​⋅(treatment×luxury)+FE+ε

In [ ]:
# Heterogeneous treatment effects by luxury: create interaction dummy
lux = df_py["luxury_bin"].to_numpy().astype(int)
treat = df_py["treatment"].to_numpy().astype(int)

# regressors: treatment and treatment*luxury
X = np.column_stack([treat, treat * lux])

# absorb date (and more if you want)
y = multiway_demean(df_py["lnprice"].to_numpy(), g_city, g_date)

X_dm = np.column_stack([
    multiway_demean(X[:,0], g_city, g_date),
    multiway_demean(X[:,1], g_city, g_date),
])

res_interact = sm.OLS(y, X_dm).fit()

res_interact.params

In [ ]:
# Only controlling for luxury (use treat and lux from above)
y_dm = multiway_demean(df_py["lnprice"].to_numpy(), g_city, g_date)

treat_dm = multiway_demean(treat, g_city, g_date)
lux_dm = multiway_demean(lux, g_city, g_date)
X_dm = np.column_stack([treat_dm, lux_dm])

res_control = sm.OLS(y_dm, X_dm).fit()

res_control.params

In [ ]:
result_chosen = res_control # res_simple, res_interact, res_control 
 
# (a) HC1 robust (Stata: , r)
res_hc1 = result_chosen.get_robustcov_results(cov_type="HC1")

# (b) Cluster by city (Stata: vce(cluster city))
res_cl_city = result_chosen.get_robustcov_results(
    cov_type="cluster",
    groups=g_city,
    use_correction=True  # small-sample correction
)

# (c) Two-way cluster (city, date) – Cameron-Gelbach-Miller
# Note: with few clusters, multiway clustering can still be noisy, but this should not return nan here.
V_2way, _, _ = cov_cluster_2groups(result_chosen, g_city, g_date)
se_2way = float(np.sqrt(V_2way[0, 0]))

print("obserevations", len(df_py))

print(f"\ntau (treatment) = {float(result_chosen.params[0]):.3f}")

print("\nHC1 robust:")
print(f"  SE = {float(res_hc1.bse[0]):.3f}, p = {float(res_hc1.pvalues[0]):.4g}")

print("\nCluster(city):")
print(f"  SE = {float(res_cl_city.bse[0]):.3f}, p = {float(res_cl_city.pvalues[0]):.4g}")

print("\nTwo-way cluster(city,date):")
t_2way = float(result_chosen.params[0]) / se_2way
print(f"  SE = {se_2way:.3f}, t = {t_2way:.3f}")

### Interpretation (for res_simple on all groups)
- estimate is positive (𝜏^=0.227→ about +25% since 𝑒 0.227 − 1 ≈ 0.255
- “significance” depends on the error structure: for group = 1
    - very significant under HC1
    - still significant under city clustering
    - borderline under two-way clustering (t ≈ 1.93)

## Archive

In [ ]:
# old two-way demean
# def twoway_demean(v, g1, g2, max_iter=200, tol=1e-10):
#     """Residualize v w.r.t. additive FE for g1 and g2 via alternating projections."""
#     v = np.asarray(v, dtype=float)
#     out = v.copy()

#     for _ in range(max_iter):
#         old = out.copy()

#         # subtract g1 means
#         m1 = np.zeros(g1.max() + 1); c1 = np.zeros(g1.max() + 1)
#         np.add.at(m1, g1, out); np.add.at(c1, g1, 1)
#         out -= (m1 / c1)[g1]

#         # subtract g2 means
#         m2 = np.zeros(g2.max() + 1); c2 = np.zeros(g2.max() + 1)
#         np.add.at(m2, g2, out); np.add.at(c2, g2, 1)
#         out -= (m2 / c2)[g2]

#         if np.max(np.abs(out - old)) < tol:
#             break

#     return out


# # Within-transform (absorb Fiexed Effects: choose which ones)
# y = twoway_demean(df_py["lnprice"].to_numpy(), g_city, g_date) 
# x = twoway_demean(df_py["treatment"].to_numpy(), g_city, g_date)

In [ ]:
# Results Hannes for all groups

# tau (treatment) = 0.227

# HC1 robust:
#   SE = 0.017, p = 4.685e-40

# Cluster(city):
#   SE = 0.053, p = 0.00358

# Two-way cluster(city,date):
#   SE = 0.118, t = 1.930

In [ ]:
    # "luxury", "luxurious", "premium", "exclusive", "elegant",
    # "boutique", "upscale", "high-end", "refined",
    # "spa", "wellness", "gourmet", "fine dining",
    # "panoramic", "rooftop", "private", "pool",
    # "designer", "bespoke",
    # "5-star", "five-star"